In [1]:
import csv
import torch
import numpy as np
import pandas as pd
import copy
import networkx as nx
import random
import matplotlib.pyplot as plt
from math import sqrt 
import os


/home/danielochana/anaconda3/envs/env3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
# Load the CSV file into a DataFrame
data = pd.read_csv("train.csv")

# Display the first five rows of the DataFrame
print(data.shape)
data.head()

(1458644, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [3]:
# Convert the pickup and dropoff timestamps to datetime objects
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [4]:
data2_5=copy.deepcopy(data)
data2_5['pickup_longitude'] = data2_5['pickup_longitude'].apply(lambda x: round(x / 0.005) * 0.005)
data2_5['pickup_latitude'] = data2_5['pickup_latitude'].apply(lambda x: round(x / 0.005) * 0.005)
data2_5['dropoff_longitude'] = data2_5['dropoff_longitude'].apply(lambda x: round(x / 0.005) * 0.005)
data2_5['dropoff_latitude'] = data2_5['dropoff_latitude'].apply(lambda x: round(x / 0.005) *0.005)


###
repeat 


In [5]:


data_all_slots = {}
slot = 0
for day in (4,5, 6):
    # Define the start and end timestamps for the day
    start_timestamp = pd.Timestamp(f'2016-01-0{day} 06:00:00')
    end_timestamp = pd.Timestamp(f'2016-01-0{day} 23:59:59')

    # Generate 6 timestamps, each with a different 3-hour timeslot
    timestamps = pd.date_range(start=start_timestamp, end=end_timestamp, freq='3H')[:6]

    # Create 15 copies of data2_5 with different timeslots
    for timestamp in timestamps:
        start_time = timestamp
        end_time = timestamp + pd.Timedelta(hours=3)
        copy = data2_5[(data2_5['pickup_datetime'] >= start_time) & (data2_5['pickup_datetime'] < end_time)].copy()
        data_all_slots[slot]=(copy)
        slot += 1


   

In [6]:
print (len(data_all_slots))

18


In [7]:
def calc_air_distance(lat1, lon1, lat2, lon2):
    # Assuming 1 degree of latitude and longitude is approximately 111 kilometers
    lat_km = 111.0
    lon_km = 90.0

    # Calculate the difference in coordinates
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1

    # Calculate the distance using Euclidean distance formula
    distance = sqrt((delta_lat * lat_km)**2 + (delta_lon * lon_km)**2)
    return distance

In [8]:
# Create a dictionary to store the directed graphs
graphs = {}

# Loop through each data in data_all_slots
for key, data in data_all_slots.items():
    # Create a directed graph
    G = nx.DiGraph()
    selfloops = 0
    # Add edges to the graph
    for row in data.itertuples():
        source = (row.pickup_longitude, row.pickup_latitude)
        destination = (row.dropoff_longitude, row.dropoff_latitude)
        distance = calc_air_distance(row.dropoff_latitude, row.dropoff_longitude, row.pickup_latitude, row.pickup_longitude)
        speed = 3600 * distance / row.trip_duration
        edge_data = {'weight': speed, 'datetime': row.pickup_datetime}
        if source != destination:
            G.add_edge(source, destination, **edge_data)
        else:
            selfloops += 1
    print(f'num of self loops in data = {selfloops}')
    # Add the graph to the dictionary
    graphs[key] = G

num of self loops in data = 12
num of self loops in data = 10
num of self loops in data = 23
num of self loops in data = 17
num of self loops in data = 20
num of self loops in data = 17
num of self loops in data = 23
num of self loops in data = 20
num of self loops in data = 12
num of self loops in data = 18
num of self loops in data = 20
num of self loops in data = 6
num of self loops in data = 9
num of self loops in data = 16
num of self loops in data = 19
num of self loops in data = 20
num of self loops in data = 17
num of self loops in data = 15


# Create a dictionary to store the directed graphs
graphs = {}

# Loop through each data in data_all_slots
for key, data in data_all_slots.items():
    # Create a directed graph
    G = nx.DiGraph()
    selfloops = 0
    # Add edges to the graph
    for row in data.itertuples():
        source = (row.pickup_longitude, row.pickup_latitude)
        destination = (row.dropoff_longitude, row.dropoff_latitude)
        distance = calc_air_distance(row.dropoff_latitude, row.dropoff_longitude, row.pickup_latitude, row.pickup_longitude)
        weight =3600*( distance /row.trip_duration )
        if source != destination:
            G.add_edge(source, destination, weight=weight)
        else :
            selfloops+=1
    print(f'num of self loops in data = {selfloops}')
    # Add the graph to the dictionary
    graphs[key] = G

In [9]:
# Create a directory to store the CSV files
os.makedirs('data_all_slots_csv/data_csv', exist_ok=True)

# Iterate over the data_all_slots dictionary
for key, data in data_all_slots.items():
    # Generate the file path
    file_path = f'data_all_slots_csv/data_{key}.csv'
    
    # Save the DataFrame as a CSV file
    data.to_csv(file_path, index=False)

In [10]:
os.makedirs('data_all_slots_csv/adj_matrices', exist_ok=True)
for key, G in graphs.items():
    A = nx.adjacency_matrix(G,  weight = 'weight')
    # Convert the matrix A to a DataFrame
    df_A = pd.DataFrame.sparse.from_spmatrix(A)
    # Save the DataFrame as a CSV file
    df_A.to_csv(f'data_all_slots_csv/adj_matrices/Adj_{key}.csv', index=True)

In [11]:
common_nodes = set(graphs[0].nodes())
for key, graph in graphs.items():
    common_nodes = common_nodes.intersection(set(graph.nodes()))

num_common_nodes = len(common_nodes)
print(num_common_nodes)

# Remove nodes that are not common to all graphs
for key, graph in graphs.items():
    graph.remove_nodes_from(set(graph.nodes()) - common_nodes)


129


In [12]:
for key, graph in graphs.items():
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()
    print(f"Graph {key}: Number of nodes = {num_nodes}, Number of edges = {num_edges}")

Graph 0: Number of nodes = 129, Number of edges = 679
Graph 1: Number of nodes = 129, Number of edges = 719
Graph 2: Number of nodes = 129, Number of edges = 729
Graph 3: Number of nodes = 129, Number of edges = 798
Graph 4: Number of nodes = 129, Number of edges = 973
Graph 5: Number of nodes = 129, Number of edges = 551
Graph 6: Number of nodes = 129, Number of edges = 723
Graph 7: Number of nodes = 129, Number of edges = 801
Graph 8: Number of nodes = 129, Number of edges = 800
Graph 9: Number of nodes = 129, Number of edges = 859
Graph 10: Number of nodes = 129, Number of edges = 948
Graph 11: Number of nodes = 129, Number of edges = 639
Graph 12: Number of nodes = 129, Number of edges = 706
Graph 13: Number of nodes = 129, Number of edges = 835
Graph 14: Number of nodes = 129, Number of edges = 822
Graph 15: Number of nodes = 129, Number of edges = 812
Graph 16: Number of nodes = 129, Number of edges = 1037
Graph 17: Number of nodes = 129, Number of edges = 738


In [13]:
import os
# Create a directory to store the CSV files
os.makedirs('data_common_slots_csv/data_csv', exist_ok=True)

# Iterate over the data_all_slots dictionary
for key, data in data_all_slots.items():
    # Generate the file path
    file_path = f'data_common_slots_csv/data_csv/data_{key}.csv'
    
    # Save the DataFrame as a CSV file
    data.to_csv(file_path, index=False)

os.makedirs('data_common_slots_csv/adj_matrices', exist_ok=True)
for key, G in graphs.items():
    A = nx.adjacency_matrix(G,  weight = 'weight')
    # Convert the matrix A to a DataFrame
    df_A = pd.DataFrame.sparse.from_spmatrix(A)
    # Save the DataFrame as a CSV file
    df_A.to_csv(f'data_common_slots_csv/adj_matrices/Adj_{key}.csv', index=True)

In [14]:
import csv

# Define the file path
file_path = 'edges_info.csv'

# Define the headers for the CSV file
headers = ['Dropoff Longitude', 'Dropoff Latitude', 'Pickup Longitude', 'Pickup Latitude', 'Speed', 'Time']

# Open the CSV file in write mode
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the headers to the CSV file
    writer.writerow(headers)
    for key, G in graphs.items():
        # Iterate over each edge in the graph
        for edge in G.edges(data=True):
            # Extract the edge information
            dropoff_longitude = edge[0][0]
            dropoff_latitude = edge[0][1]
            pickup_longitude = edge[1][0]
            pickup_latitude = edge[1][1]
            speed = edge[2]['weight']
            time = edge[2]['datetime']
        
            # Write the edge information to the CSV file
            writer.writerow([dropoff_longitude, dropoff_latitude, pickup_longitude, pickup_latitude, speed, time])

print(f"CSV file '{file_path}' has been created.")

CSV file 'edges_info.csv' has been created.


In [15]:
non_directed_graphs = {}

# Loop through each graph in the graphs dictionary
for key, graph in graphs.items():
    # Create a non-directed version of the graph
    non_directed_graph = graph.to_undirected()
    
    # Add the non-directed graph to the dictionary
    non_directed_graphs[key] = non_directed_graph

In [16]:
# Loop through each graph in the non_directed_graphs dictionary
for key, graph in non_directed_graphs.items():
    # Loop through each edge in the graph
    for edge in graph.edges(data=True):
        # Set the weight of the edge to 1
        edge[2]['weight'] = 1

In [17]:
import os

# Create a directory to store the non-directed graph information
os.makedirs('non_directed_graph_info', exist_ok=True)

# Save the adjacency matrix of all graphs
for key, graph in non_directed_graphs.items():
    A = nx.adjacency_matrix(graph, weight='weight')
    # Convert the matrix A to a DataFrame
    df_A = pd.DataFrame.sparse.from_spmatrix(A)
    # Save the DataFrame as a CSV file
    df_A.to_csv(f'non_directed_graph_info/Adj_{key}.csv', index=True)

In [24]:
for key, graph in non_directed_graphs.items():
    A = nx.adjacency_matrix(graph, weight='weight')
    df_A = pd.DataFrame.sparse.from_spmatrix(A)
    print (df_A.shape)
    

(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)
(129, 129)


In [19]:
import csv

 
# Define the file path for the edges_info.csv file
edges_file_path = 'non_directed_graph_info/ID_edges_info.csv'

# Define the headers for the CSV file
headers = ['ID1', 'ID2', 'datetime']

# Open the CSV file in write mode
with open(edges_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write the headers to the CSV file
    writer.writerow(headers)
    
    # Assign a unique ID to each node
    node_ids = {node: i+1 for i, node in enumerate(common_nodes)}
    
    # Loop through each graph in the non_directed_graphs dictionary
    for key, graph in non_directed_graphs.items():
        # Loop through each edge in the graph
        for edge in graph.edges(data=True):
            # Extract the edge information
            node1 = edge[0]
            node2 = edge[1]
            datetime = edge[2]['datetime']
            
            # Get the unique IDs for the nodes
            id1 = node_ids[node1]
            id2 = node_ids[node2]
            
            # Write the edge information to the CSV file
            writer.writerow([id1, id2, datetime])

print(f"CSV file '{edges_file_path}' has been created.")

CSV file 'non_directed_graph_info/ID_edges_info.csv' has been created.
